In [1]:
%matplotlib inline

from functools import partial

import dask
from dask import delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cmlreaders import CMLReader, get_data_index
from cmlreaders.timeseries import TimeSeries

from thetamod import connectivity, tmi
from thetamod.pipeline import TMIPipeline

/Users/depalati/miniconda3/envs/thetamod/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
subject = "R1286J"
experiment = "catFR3"
session = 0
rootdir = "~/mnt/rhino"

In [3]:
pipeline = TMIPipeline(subject, experiment, session, rootdir)

In [4]:
pipeline.run()

TypeError: Delayed objects of unspecified length are not iterable

fmin=4.000 Hz corresponds to 4.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.250 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.


/Users/depalati/src/thetamod/thetamod/connectivity.py:144: RuntimeWarning: fmin=4.000 Hz corresponds to 4.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.250 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.
  verbose=False)


In [5]:
from cmlreaders.path_finder import PathFinder

In [6]:
finder = PathFinder("R1111M", "FR1", 0, rootdir="~/mnt/rhino")

In [10]:
with open(finder.find("pairs")) as f:
    print(f.read(2000))

{
  "R1111M": {
    "code": "R1111M", 
    "id": "r1.111.0_0", 
    "pairs": {
      "LAS1-LAS2": {
        "atlases": {
          "avg": {
            "id": "avg", 
            "region": "inferiortemporal", 
            "x": -42.61, 
            "y": 1.125, 
            "z": -47.255
          }, 
          "avg.dural": {
            "id": "avg.dural", 
            "region": "inferiortemporal", 
            "x": -40.325014, 
            "y": -0.977092, 
            "z": -42.874617
          }, 
          "dk": {
            "id": "dk", 
            "region": "inferiortemporal", 
            "x": null, 
            "y": null, 
            "z": null
          }, 
          "ind": {
            "id": "ind", 
            "region": "inferiortemporal", 
            "x": -35.445, 
            "y": 13.665, 
            "z": -61.24
          }, 
          "ind.dural": {
            "id": "ind.dural", 
            "region": "inferiortemporal", 
            "x": -35.617608, 
            "y": 14.1